## Homework

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-11 18:48:24--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 74.125.134.100, 74.125.134.138, 74.125.134.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.134.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hpftm73uj02i3ug2vlb1oeglqm1ffqkl/1652294850000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-11 18:48:26--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hpftm73uj02i3ug2vlb1oeglqm1ffqkl/1652294850000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 142.250.97.132, 2607:f8b0:400c:c18::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.goog

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 292 entries, id to price
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 44.6+ MB


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [7]:
df.isna().sum()

id                       0
timestamp                0
full_sq                  0
life_sq               4103
floor                  113
                      ... 
mosque_count_5000        0
leisure_count_5000       0
sport_count_5000         0
market_count_5000        0
price                    0
Length: 292, dtype: int64

In [8]:
df.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


id                    9.999500e+03
full_sq               5.425995e+01
life_sq               3.403233e+01
floor                 7.644441e+00
max_floor             1.260393e+01
                          ...     
mosque_count_5000     4.431000e-01
leisure_count_5000    8.626950e+00
sport_count_5000      5.305495e+01
market_count_5000     6.005700e+00
price                 7.118799e+06
Length: 276, dtype: float64

In [9]:
len(df.columns)

292

Удалим строки, где более 30% значений в колонках NaN, потому что они могут быть недостаточно информативными.
P.S. видимо, таких вообще не оказалось)

In [10]:
df.dropna(thresh=len(df.columns)*0.7)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


In [11]:
df.fillna(df.mean(), inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [12]:
df.isna().sum().sum()

0

In [13]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [c for c in df.columns if df[c].dtype.name=='object']
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [14]:
t = 0
for column in cat_columns:
  # df[column].value_counts().plot(kind='bar')
  print(f'{t}) ', column, '\n', df[column].value_counts())
  t+=1

0)  product_type 
 Investment       12840
OwnerOccupier     7160
Name: product_type, dtype: int64
1)  sub_area 
 Poselenie Sosenskoe               1126
Nekrasovka                        1074
Poselenie Vnukovskoe               859
Poselenie Moskovskij               603
Poselenie Voskresenskoe            464
                                  ... 
Molzhaninovskoe                      2
Poselenie Shhapovskoe                2
Poselenie Kievskij                   2
Poselenie Mihajlovo-Jarcevskoe       1
Poselenie Klenovskoe                 1
Name: sub_area, Length: 146, dtype: int64
2)  culture_objects_top_25 
 no     18748
yes     1252
Name: culture_objects_top_25, dtype: int64
3)  thermal_power_plant_raion 
 no     18912
yes     1088
Name: thermal_power_plant_raion, dtype: int64
4)  incineration_raion 
 no     18452
yes     1548
Name: incineration_raion, dtype: int64
5)  oil_chemistry_raion 
 no     19809
yes      191
Name: oil_chemistry_raion, dtype: int64
6)  radiation_raion 
 no     127

In [15]:
from sklearn.preprocessing import LabelEncoder
yes_no_encoder = None
for column in cat_columns:
  counted = df[column].unique()

  if 'yes' in counted or 'no' in counted:
    if yes_no_encoder == None:
      yes_no_encoder = LabelEncoder()
      yes_no_encoder.fit(df[column])
    df[column] = yes_no_encoder.transform(df[column])
  elif column == 'ecology':
    def convert(x):
      if x == 'no data':
        return 0
      elif x == 'poor':
        return 1
      elif x == 'satisfactory':
        return 2
      elif x == 'good':
        return 3
      elif x == 'excellent':
        return 4
    df[column] = df[column].apply(convert)
  else:
    temp_encoder = LabelEncoder()
    temp_encoder.fit(df[column])
    df[column] = temp_encoder.transform(df[column])    

In [16]:
t = 0
for column in cat_columns:
  # df[column].value_counts().plot(kind='bar')
  print(f'{t}) ', column, '\n', df[column].value_counts())
  t+=1

0)  product_type 
 0    12840
1     7160
Name: product_type, dtype: int64
1)  sub_area 
 102    1126
72     1074
103     859
94      603
105     464
       ... 
66        2
100       2
88        2
93        1
89        1
Name: sub_area, Length: 146, dtype: int64
2)  culture_objects_top_25 
 0    18748
1     1252
Name: culture_objects_top_25, dtype: int64
3)  thermal_power_plant_raion 
 0    18912
1     1088
Name: thermal_power_plant_raion, dtype: int64
4)  incineration_raion 
 0    18452
1     1548
Name: incineration_raion, dtype: int64
5)  oil_chemistry_raion 
 0    19809
1      191
Name: oil_chemistry_raion, dtype: int64
6)  radiation_raion 
 0    12767
1     7233
Name: radiation_raion, dtype: int64
7)  railroad_terminal_raion 
 0    19242
1      758
Name: railroad_terminal_raion, dtype: int64
8)  big_market_raion 
 0    18182
1     1818
Name: big_market_raion, dtype: int64
9)  nuclear_reactor_raion 
 0    19426
1      574
Name: nuclear_reactor_raion, dtype: int64
10)  detention_faci

In [17]:
df = df.drop(columns=['id','timestamp'])

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

In [18]:
from sklearn.model_selection import train_test_split
y = df['price']
X = df.drop(columns=['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [19]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=42)

Проверьте качество на отложенной выборке.

In [20]:
from sklearn.model_selection import cross_val_score
# R^2
regressor.score(X_test, y_test)

0.5143019848499304

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [21]:
df_orig = pd.read_csv("data.csv", parse_dates=["timestamp"])

month_year = (df_orig.timestamp.dt.month + df_orig.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df_orig.timestamp.dt.weekofyear + df_orig.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [23]:
month = df_orig.timestamp.dt.month
df["month"] = month

week = df_orig.timestamp.dt.week
df["week"] = week


df["floor_ratio"] = df_orig.floor/df_orig.max_floor

df["square_ratio"] = df_orig.kitch_sq/df_orig.full_sq

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [41]:
import numpy as np
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.fillna(df.median())
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [42]:
X_train.isna().sum().sum()

0

In [46]:
from sklearn.model_selection import GridSearchCV

n_features = X.shape[1]
n_samples = X.shape[0]

param_grid = param_grid ={
                    'max_depth': range(1,15,2),
                    'min_samples_leaf':[1, 0.3,0.5, n_samples//2, n_samples//3, n_samples//5]}
regr = DecisionTreeRegressor(random_state=42)
g_search = GridSearchCV(regr, cv=3, n_jobs=-1, verbose=5, param_grid = param_grid)
g_search.fit(X_train, y_train)

print(g_search.best_params_, g_search.best_score_)

Fitting 3 folds for each of 42 candidates, totalling 126 fits
{'max_depth': 11, 'min_samples_leaf': 1} 0.9954739260151545


In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {'max_depth': [5, 10, None], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}
rfr = RandomForestRegressor(random_state = 42)

g_search = GridSearchCV(estimator = rfr, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)
g_search.fit(X_train, y_train);

print(g_search.best_params_, g_search.best_score_)

In [ ]:
!pip install catboost
%%time
from catboost import CatBoostRegressor


model = CatBoostRegressor()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid,
                                       X=X_train,
                                       y=y_train,
                                       plot=True)

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
%%time
df_investment = df[df.product_type == 0]
df_owneroccupier = df[df.prodcut_type == 1]

invest_train_X, invest_train_Y, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
owner_train_X, owner_train_Y, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)

model_invest = CatBoostRegressor()
model_owner = CatBoostRegressor()

model_invest.fit(invest_train_X, invest_train_Y)
print("Investment R^2: ", model_invest.score(X_test, y_test))

model_owner.fit(owner_train_X, owner_train_Y)
print("Owner Occupier R^2: ", model_owner.score(X_test, y_test))

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).